In [1]:
import igl
import meshplot as mp
import numpy as np

# Introduction and shrinking operation

In [2]:
v, t, f = igl.read_mesh('data/hand.mesh')
mp.plot(v, f)

/home/rage/anaconda3/envs/geo_computing/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, -0.0…

In [9]:
# you can reshape the points and faces like this and you get the same mesh
v[t].reshape(-1, 3), np.arange(len(t)*4).reshape(-1, 4)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, -0.0…

In [12]:
def shrink(v, t):
    alpha = 0.5
    bc = igl.barycenter(v, t) # v[t].mean(axis=1)
    bc = bc.reshape(-1, 1, 3)
    return (v[t] - bc) * 0.5 + bc

mp.plot(shrink(v, t).reshape(-1, 3), np.arange(len(t)*4).reshape(-1, 4)) # this is the same plot of below!

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000123…

# Boundary condtions

In [22]:
import ipywidgets as iw

# We now use meshzoo to generate a simple sphere we will use to build boundary conditions
import meshzoo
v_s, f_s = meshzoo.octa_sphere(5)

# Another thing you generally want to in this kind of applications is to "normalise" the position
# of your mesh
v -= v.min(axis=0)
v /= v.max()

In [23]:
paint_ui = mp.plot(v, f)
paint_ui.add_mesh(0.1*v_s, f_s, c=np.array([0, 1, 0]),
                  shading=dict(flat=False))

# move the sphere where you like, press record and bang you have your boundary condition
recording = []
button = iw.Button(description='Record sphere position')

def button_click(b):
    recording.append(sphere.coord)
button.on_click(button_click)
display(button)
    
def sphere(rad, x, y, z):
    paint_ui.update_object(oid=1, vertices=v_s*rad + np.array([x, y, z]))
    sphere.coord = [rad, x, y, z]

# to move the sphere with sliders
mp.interact(sphere,
        rad = iw.FloatSlider(min=0.01, max=1, value=0.1),
        x = iw.FloatSlider(min=0.01, max=1, value=0.1),
        y = iw.FloatSlider(min=0.01, max=1, value=0.1),
        z = iw.FloatSlider(min=0.01, max=1, value=0.1)
           )


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3685695…

Button(description='Record sphere position', style=ButtonStyle())

interactive(children=(FloatSlider(value=0.1, description='rad', max=1.0, min=0.01), FloatSlider(value=0.1, des…

<function __main__.sphere(rad, x, y, z)>

In [24]:
recording

[[0.21, 0.41, 0.1, 0.1], [0.11, 0.31, 0.91, 0.21]]

In [36]:
import polyfempy as pf

In [37]:
solver = pf.Solver()
solver.set_mesh(v, t)

[2021-10-15 09:07:49.167] [polyfem] [info] Loading mesh...
[2021-10-15 09:07:49.167] [polyfem] [info] mesh bb min [0, 0, 0], max [0.737139, 1, 0.36557]
[2021-10-15 09:07:49.168] [polyfem] [info]  took 0.00096s


In [40]:
# Set the boundary condition
def sideset(p):
    for i, (rad, x, y, z) in enumerate(recording, 2):
        if np.linalg.norm(p - [x, y, z]) < rad: # check if the point is inside the sphere
            return i # index of boundary condition
    return 1 # index of mesh inside boundary condition

solver.set_boundary_side_set_from_bary(sideset)

In [42]:
# Plot boundary conditons
vi, fi, ci = solver.get_boundary_sidesets()
mp.plot(vi, fi, c=ci)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3685695…

In [44]:
# Initialise the problem / set up settings
problem = pf.Problem()
problem.set_displacement(2, [0, 0, 0])
problem.set_force(3, [1, 0, 0])

settings = pf.Settings() # default settings
settings.set_pde(pf.PDEs.LinearElasticity)
settings.set_material_params('E', 2e3) # young's modulus
settings.set_material_params('nu', 0.4) # Poisson ration
settings.set_problem(problem)

In [46]:
solver.settings(settings) # feed the settings
solver.solve()

[2021-10-15 09:12:01.120] [polyfem] [info] simplex_count: 	29998
[2021-10-15 09:12:01.120] [polyfem] [info] regular_count: 	0
[2021-10-15 09:12:01.120] [polyfem] [info] regular_boundary_count: 	0
[2021-10-15 09:12:01.120] [polyfem] [info] simple_singular_count: 	0
[2021-10-15 09:12:01.120] [polyfem] [info] multi_singular_count: 	0
[2021-10-15 09:12:01.120] [polyfem] [info] boundary_count: 	0
[2021-10-15 09:12:01.120] [polyfem] [info] multi_singular_boundary_count: 	0
[2021-10-15 09:12:01.120] [polyfem] [info] non_regular_count: 	0
[2021-10-15 09:12:01.120] [polyfem] [info] non_regular_boundary_count: 	0
[2021-10-15 09:12:01.120] [polyfem] [info] undefined_count: 	0
[2021-10-15 09:12:01.120] [polyfem] [info] total count:	 29998
[2021-10-15 09:12:01.120] [polyfem] [info] Building isoparametric basis...
[2021-10-15 09:12:01.184] [polyfem] [info] Computing polygonal basis...
[2021-10-15 09:12:01.184] [polyfem] [info]  took 1.4e-05s
[2021-10-15 09:12:01.189] [polyfem] [info] hmin: 0.0010579

In [47]:
# the solution samples some points in the element space hence we will get some disconnected triangles
sol_pts, sol_tri, disp = solver.get_sampled_solution()

In [49]:
mp.plot(sol_pts+disp, sol_tri)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3698654…

In [55]:
# sometimes it's hard to see the result of a simulation. We can help ourself by using colors

# We get the von mises stresses 
vonmises, _ = solver.get_sampled_mises_avg()

_, unid, univ = np.unique((sol_pts*1e8).astype(int), axis=0,
                         return_index=True, return_inverse=True)

# we can now display the stress
mp.plot(sol_pts[unid] + disp[unid], univ[sol_tri], c = vonmises[unid])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3698654…

In [51]:
? solver.get_sampled_mises_avg
